# Training

### Imports

In [ ]:
from dimgpt import utils
from dimgpt.training.datasets import *
from dimgpt.training.model import Model
from dimgpt.training.trainer import Trainer
from dimgpt.data.tokenizer import *
from dimgpt.settings import *

utils.reset_rand()

### Check GPU

In [ ]:
utils.check_gpu()

### Tokenizer

In [ ]:
tokenizer = Tokenizer()

print(f'Vocab size: {len(tokenizer.vocab):,}\n')

for v in tokenizer.vocab:
	print(f'[{v}]', end = ' ')

### Dataset

In [ ]:
dataset = PretrainingDataset(tokenizer)
#dataset = FinetuningDataset(tokenizer)

### Model

In [ ]:
model = Model().to(DEVICE)
model.summary()

In [ ]:
x, y, strength = dataset.next_train()

print(f'Batch shape: {tuple(x.shape)}\n')

print(tokenizer.decode(x[0]))

del x, y, strength

### Training

In [ ]:
trainer = Trainer(model, dataset)
trainer.find_previous_session()

trainer.train()